# Aula 5 - Solução dos exercícios - Parte 2
Leandro Carísio Fernandes

<br>

Treinar um modelo seq2seq (a partir do T5-base) na tarefa de expansão de documentos.

- Usar como treino o dataset "tiny" do MS MARCO na tarefa doc2query
https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv
- doc2query: A entrada é a passagem e o target é a query
- Note que apenas pares (query, passagem relevante) são usados como treino.
- O treino é relativamente rápido (<1 hora).
- Validar a cada X steps usando o sacreBLEU 
- A parte lenta deste exercício é a pré-indexação: para cada documento da coleção, temos que gerar uma ou mais queries, que depois são concatenadas ao documento original, e esse documento "expandido" é indexado.
- Avaliar no TREC-COVID (171K docs), pois é menor que o MS MARCO/TREC-DL 2020 (8.8M passagens). 
 - Indice invertido do Trec-covid no pyserini: beir-v1.0.0-trec-covid-flat
 - Corpus e queries na HF: https://huggingface.co/datasets/BeIR/trec-covid
 - qrels: https://huggingface.co/datasets/BeIR/trec-covid-qrels
 - Usar nDCG@10
 - Comparar com o BM25 com e sem os documentos expandidos pelo doc2query


<br>

Esse caderno contém apenas a parte 2 do exercício, o fine-tuning do T5 usando MS MARCO e geração das queries expandidas.

## Preparação do ambiente

In [ ]:
# Estourou o disco e eu não consegui nem rodar o ls sem fazer isso:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

### Variáveis


In [ ]:
# Link de download do dataset MS-MARCO reduzido
url_tiny_msmarco = 'https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv'

# Link de download da base TREC-COVID. O link foi retirado de https://huggingface.co/datasets/BeIR/trec-covid
url_trec_covid = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

# Dados para treinamento do modelo
nome_modelo = "t5-base"
treinar_modelo = False
batch_size = 8
steps = 50
epochs = 100

# Dados para recuperar o modelo
carregar_modelo_tunado = True
url_modelo_salvo = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/modelo_doc_relevante_final/'

# Dados para inferência
expandir_docs = False # Já foi expandido, apenas carrega
carregar_arquivo_docs_queries_expandidas = True
inferir_docs_a_partir_do_indice = 171328
batch_size_inferencia = 16
numero_queries_geradas = 10
arquivo_docs_queries_expandidas = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/doc_com_queries_expandidas.pickle'

### Instalação de libs e montagem do Drive

In [ ]:
%%time
# Já monta o drive, pois vamos usar o índice invertido da Aula 1 para usar o BM25 implementado também na aula 1
# Além disso, é necessário para salvar/recuperar o modelo tunado
from google.colab import drive
drive.mount('/content/drive')

# Instala libs
!pip install faiss-cpu
!pip install transformers datasets
!pip install sacrebleu
!pip install evaluate
!pip install sentencepiece

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
CPU times: user 207 ms, sys: 264 ms, total: 471 ms
Wall time: 29.3 s


### Download do MS-MARCO tiny

In [ ]:
from pathlib import Path
# Só faz o download se ainda não tiver feito
if not Path('./collections/msmarco_triples.train.tiny.tsv').is_file():
  !wget {url_tiny_msmarco} -P collections # type: ignore

## Fine-tuning

Tive dificuldades com esse treinamento. O código de treinamento foi copiado do caderno da Monique.

### Carrega modelo e tokenizador:

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = T5Tokenizer.from_pretrained(nome_modelo)
model = T5ForConditionalGeneration.from_pretrained(url_modelo_salvo).to(device) if carregar_modelo_tunado else T5ForConditionalGeneration.from_pretrained(nome_modelo).to(device)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


### Criação dos dadasets de treino e validação

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from datasets import Dataset

# Cria dataframes 

msmarco_df = pd.read_csv("collections/msmarco_triples.train.tiny.tsv", sep='\t', names=['query', 'relevante', 'nao_relevante'], header=None)
msmarco_train_df, msmarco_val_df = train_test_split(msmarco_df, test_size=0.1, random_state=42)

In [ ]:
from torch.utils import data

class MsMarcoDataset(data.Dataset):
  def __init__(self, dataframe, tokenizer):
    self.df = dataframe
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    tokenized_input = self.tokenizer(self.df.iloc[index]['relevante'])
    tokenized_query = self.tokenizer(self.df.iloc[index]['query'])
    return {"input_ids": tokenized_input["input_ids"], 
            "attention_mask": tokenized_input["attention_mask"], 
            "labels": tokenized_query["input_ids"]}


# Cria datasets
msmarco_train_ds = MsMarcoDataset(msmarco_train_df, tokenizer)
msmarco_val_ds = MsMarcoDataset(msmarco_val_df, tokenizer)

### Treino do modelo

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, T5ForConditionalGeneration
import numpy as np
import evaluate
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    return result

def treina_modelo_e_salva():
  training_args = Seq2SeqTrainingArguments(output_dir=f"/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/modelo_doc_relevante",
                                            overwrite_output_dir=True,
                                            per_device_train_batch_size=batch_size,
                                            per_device_eval_batch_size=batch_size,
                                            gradient_accumulation_steps=8,
                                            evaluation_strategy='steps',
                                            eval_steps=steps, logging_steps=steps, 
                                            save_steps=steps,
                                            predict_with_generate=True,
                                            fp16=True, 
                                            num_train_epochs=100,
                                            load_best_model_at_end=True,
                                            metric_for_best_model='bleu',
                                            save_total_limit = 2
                                          )
  #Se não usar o collator e tokenizar com parâmetros além da entrada, todo tipo de erro acontece.
  data_collator = DataCollatorForSeq2Seq(
      tokenizer,
      model=model,
      label_pad_token_id=-100,
      pad_to_multiple_of=8 if training_args.fp16 else None,
  )

  trainer = Seq2SeqTrainer(model=model,
                          args=training_args,
                          train_dataset=msmarco_train_ds,
                          eval_dataset=msmarco_val_ds,
                          data_collator=data_collator,
                          tokenizer=tokenizer,
                          compute_metrics=compute_metrics
                          )

  train_results = trainer.train()
  model.save_pretrained(url_modelo_salvo)

if treinar_modelo:
  treina_modelo_e_salva()

## Extração de queries

Download do TREC-COVID:

In [ ]:
from pathlib import Path
import hashlib

# Baixa o trec-covid.zip
if not Path('./collections/trec-covid.zip').is_file():
  # O arquivo ainda não foi baixado. Verifica se está no drive:
  if Path('/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/trec-covid.zip').is_file():
    !mkdir -p './collections/' && cp '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/trec-covid.zip' './collections/trec-covid.zip'  # type: ignore
    !unzip -o collections/trec-covid.zip -d ./collections # type: ignore
  else:
    !wget {url_trec_covid} -P collections # type: ignore
    !mkdir -p '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/' && cp './collections/trec-covid.zip' '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula5-t5-doc2query/'
    !unzip -o collections/trec-covid.zip -d ./collections # type: ignore

Extrai todos os documentos do TREC-COVID e coloca em um dict:

In [ ]:
%%time
from collections import OrderedDict
import json
import pickle

trec_covid_docs = OrderedDict({})

if carregar_arquivo_docs_queries_expandidas:
  with open(arquivo_docs_queries_expandidas, 'rb') as f:
    trec_covid_docs = pickle.load(f)
else:
  with open('./collections/trec-covid/corpus.jsonl') as file:
    for line in file:
      doc = json.loads(line)
      trec_covid_docs[doc['_id']] = {"doc_original": f"{doc['title']} {doc['text']}"}

CPU times: user 605 ms, sys: 174 ms, total: 779 ms
Wall time: 977 ms


In [ ]:
%%time

def predict(texts, num_return_sequences=5):
  input_ids = tokenizer(texts, return_tensors="pt", padding="max_length", truncation=True).input_ids.to(device)
  sequence_ids = model.generate(input_ids, do_sample=True, top_p=0.9, max_new_tokens=128, num_return_sequences=num_return_sequences)
  sequences = tokenizer.batch_decode(sequence_ids, skip_special_tokens=True)
  return sequences

# Teste pra tamanho do batch
#texto = trec_covid_docs['ug7v899j']['doc_original']
#retorno = predict([texto, texto, texto, texto], 50)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.15 µs


In [ ]:
from tqdm.auto import tqdm
import pickle
import json
import torch
torch.manual_seed(42)

!nvidia-smi

Tue Apr 11 17:25:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    30W /  70W |  11707MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def expande_docs_e_salva():
  # Recupera a lista de ids dos documentos, que é a key de trec_covid_docs
  ids_docs = list(trec_covid_docs.keys())

  # Faz o loop em todos os documentos, processando batch_size_inferencia de cada vez
  for i in tqdm(range(inferir_docs_a_partir_do_indice, len(ids_docs), batch_size_inferencia)):
    # Para cada batch, recupera a lista de ids que o batch está tratando ...
    ids = ids_docs[i:i+batch_size_inferencia]
    # ... e o documento original que deve ser expandido
    texts = [trec_covid_docs[id]['doc_original'] for id in ids]

    # Processa as queries...
    queries_batch = predict(texts, num_return_sequences=numero_queries_geradas)
    
    # Processou em batch, então o resultado também é em batch. Tem que desagrupar:
    for j in range(0, len(ids)*numero_queries_geradas, numero_queries_geradas):
      # Recupera a lista de queries para a id (tratamos numero_queries_geradas id's por batch)
      id = int(j/numero_queries_geradas)
      queries_para_id = queries_batch[j:j+numero_queries_geradas]
      # E popula o arquivo trec_covid_docs com as queries expandidas
      trec_covid_docs[ids[id]]['query_expandida'] = " ".join(queries_para_id)

    # Vamos salvar o arquivo a cada 300 batchs processados
    if i % (300*batch_size_inferencia) == 0:
      with open(arquivo_docs_queries_expandidas, 'wb') as f:
        pickle.dump(trec_covid_docs, f)
      print(f'Arquivo salvo. i = {i}')
  # Salva o arquivo completo
  with open(arquivo_docs_queries_expandidas, 'wb') as f:
    pickle.dump(trec_covid_docs, f)
  print(f'Arquivo salvo. i = {i}')

if expandir_docs:
  expande_docs_e_salva()

  0%|          | 0/209 [00:00<?, ?it/s]

Arquivo salvo. i = 168000
Arquivo salvo. i = 171328
